Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Want to *actually* do machine learning? 
## Part 2: Build and train models

*Made for Microsoft Build 2019*

This is the second in a series that walks through how Azure Machine Learning service can speed up your machine learning modelling workflow so you can focus on the interesting tasks that matter. 

**Goal:**
In this notebook, we'll build and train regression models using the data we've prepared in Part 1. 


In [ ]:
!source activate py36 && pip install seaborn

### Imports

In [ ]:
%matplotlib inline
import pathlib
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 8, 6
import pandas as pd
import numpy as np
import seaborn as sns
import os
from azureml.core.model import Model
pd.set_option('display.max_columns', 500)
from collections import defaultdict

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

pd.options.mode.chained_assignment = None

from torch.nn import init
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch.optim import lr_scheduler

device = torch.device("cpu")
device

from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')

### Helper Functions

In [ ]:
def separate_y(df):
    y = np.log(df.cost)
    df.drop(['cost'], axis=1, inplace=True)
    return df, y

def split_features(df):
    catf = ['pickup_year', 'pickup_month', 'pickup_monthday', 'pickup_weekday', 
            'pickup_hour', 'pickup_minute', 'pickup_second', 'vendor', 'passengers']

    numf = [col for col in df.columns if col not in catf]
    for c in catf: 
        df[c] = df[c].astype('category').cat.as_ordered()
        df[c] = df[c].cat.codes+1
    
    return catf, numf

def numericalize(df):
    df[name] = col.cat.codes+1

def split_dataset(df, y): return train_test_split(df, y, test_size=0.25, random_state=42)

def inv_y(y): return np.exp(y)

def get_numf_scaler(train): return preprocessing.StandardScaler().fit(train)

def scale_numf(df, num, scaler):
    cols = numf
    index = df.index
    scaled = scaler.transform(df[numf])
    scaled = pd.DataFrame(scaled, columns=cols, index=index)
    return pd.concat([scaled, df.drop(numf, axis=1)], axis=1)

class RegressionColumnarDataset(data.Dataset):
    def __init__(self, df, cats, y):
        self.dfcats = df[cats]
        self.dfconts = df.drop(cats, axis=1)
        
        self.cats = np.stack([c.values for n, c in self.dfcats.items()], axis=1).astype(np.int64)
        self.conts = np.stack([c.values for n, c in self.dfconts.items()], axis=1).astype(np.float32)
        self.y = y.values.astype(np.float32)
        
    def __len__(self): return len(self.y)

    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx], self.y[idx]]
    
def rmse(targ, y_pred):
    return np.sqrt(mean_squared_error(inv_y(y_pred), inv_y(targ))) #.detach().numpy()

def emb_init(x):
    x = x.weight.data
    sc = 2/(x.size(1)+1)
    x.uniform_(-sc,sc)

class MixedInputModel(nn.Module):
    def __init__(self, emb_szs, n_cont, emb_drop, out_sz, szs, drops, y_range, use_bn=True):
        super().__init__()
        for i,(c,s) in enumerate(emb_szs): print(emb_szs[i],c,s)
        for i,(c,s) in enumerate(emb_szs): assert c >= 1,'cardinality must be >=2, got emb_szs[{i}]: ({c},{s})'
        self.embs = nn.ModuleList([nn.Embedding(c, s) for c,s in emb_szs])
        for emb in self.embs: emb_init(emb)
        n_emb = sum(e.embedding_dim for e in self.embs)
        self.n_emb, self.n_cont=n_emb, n_cont
        
        szs = [n_emb+n_cont] + szs
        self.lins = nn.ModuleList([nn.Linear(szs[i], szs[i+1]) for i in range(len(szs)-1)])
        self.bns = nn.ModuleList([nn.BatchNorm1d(sz) for sz in szs[1:]])
        for o in self.lins: nn.init.kaiming_normal_(o.weight.data)
        self.outp = nn.Linear(szs[-1], out_sz)
        nn.init.kaiming_normal_(self.outp.weight.data)

        self.emb_drop = nn.Dropout(emb_drop)
        self.drops = nn.ModuleList([nn.Dropout(drop) for drop in drops])
        self.bn = nn.BatchNorm1d(n_cont)
        self.use_bn,self.y_range = use_bn,y_range

    def forward(self, x_cat, x_cont):
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embs)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x2 = self.bn(x_cont)
            x = torch.cat([x, x2], 1) if self.n_emb != 0 else x2
        for l,d,b in zip(self.lins, self.drops, self.bns):
            x = F.relu(l(x))
            if self.use_bn: x = b(x)
            x = d(x)
        x = self.outp(x)
        if self.y_range:
            x = torch.sigmoid(x)
            x = x*(self.y_range[1] - self.y_range[0])
            x = x+self.y_range[0]
        return x.squeeze()

def fit(model, train_dl, val_dl, loss_fn, opt, scheduler, epochs=3):
    num_batch = len(train_dl)
    for epoch in tnrange(epochs):      
        y_true_train = list()
        y_pred_train = list()
        total_loss_train = 0          
        
        t = tqdm_notebook(iter(train_dl), leave=False, total=num_batch)
        for cat, cont, y in t:
            cat = cat
            cont = cont
            y = y
            
            t.set_description('Epoch {epoch}')
            
            opt.zero_grad()
            pred = model(cat, cont)
            loss = loss_fn(pred, y)
            loss.backward()
            lr[epoch].append(opt.param_groups[0]['lr'])
            tloss[epoch].append(loss.item())
            scheduler.step()
            opt.step()
            
            t.set_postfix(loss=loss.item())
            
            y_true_train += list(y.cpu().data.numpy())
            y_pred_train += list(pred.cpu().data.numpy())
            total_loss_train += loss.item()
            
        train_acc = rmse(y_true_train, y_pred_train)
        train_loss = total_loss_train/len(train_dl)
        
        if val_dl:
            y_true_val = list()
            y_pred_val = list()
            total_loss_val = 0
            for cat, cont, y in tqdm_notebook(val_dl, leave=False):
                cat = cat
                cont = cont
                y = y
                pred = model(cat, cont)
                loss = loss_fn(pred, y)
                
                y_true_val += list(y.cpu().data.numpy())
                y_pred_val += list(pred.cpu().data.numpy())
                total_loss_val += loss.item()
                vloss[epoch].append(loss.item())
            valacc = rmse(y_true_val, y_pred_val)
            valloss = total_loss_val/len(valdl)
            print('Epoch {epoch}: train_loss: {train_loss:.4f} train_rmse: {train_acc:.4f} | val_loss: {valloss:.4f} val_rmse: {valacc:.4f}')
        else:
            print('Epoch {epoch}: train_loss: {train_loss:.4f} train_rmse: {train_acc:.4f}')
    
    return lr, tloss, vloss

# Preparing the data

Tha data is a half-million random sample from the 55M original training set from the [New York City Taxi Fare Prediction](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data) Kaggle's challenge

In [ ]:
from azureml.core import Workspace, Dataset 

ws = Workspace.from_config()
dataset = Dataset.get(workspace = ws, name='nyc_taxi_clean')
df = dataset.to_pandas_dataframe()

In [ ]:
df_new = df
df_new["pickup_datetime"] = pd.to_numeric(df_new["pickup_datetime"])
df_new["dropoff_datetime"] = pd.to_numeric(df_new["dropoff_datetime"])
df_new["pickup_weekday"] = pd.to_datetime(df_new["pickup_weekday"], errors = 'coerce')
df_new['pickup_weekday'] = df_new['pickup_weekday'].dt.weekday
df_new["dropoff_weekday"] = pd.to_datetime(df_new["dropoff_weekday"], errors = 'coerce')
df_new['dropoff_weekday'] = df_new['dropoff_weekday'].dt.weekday
df_new=df_new.fillna("1")
df=df_new

In [ ]:
df.columns

In [ ]:
df.passengers.describe()

df.passengers.quantile([.85, .99])

In [ ]:
df.cost.describe()

df.cost.quantile([.85, .99])

In [ ]:
df = df.loc[(df.cost > 0) & (df.passengers < 6) & (df.cost < 53),:]
df.shape
df.head()

In [ ]:
df, y = separate_y(df)

In [ ]:
ax = y.hist(bins=20, figsize=(8,6))
_ = ax.set_xlabel("Ride Value (EUR)")
_ = ax.set_ylabel("# Rides")
_ = ax.set_title('Ditribution of Ride Values (USD)')

In [ ]:
catf, numf = split_features(df)

len(catf)
catf

len(numf)
numf

In [ ]:
df.head()

In [ ]:
X_train, X_test, y_train, y_test = split_dataset(df, y)

X_train.shape
X_test.shape

In [ ]:
scaler = get_numf_scaler(X_train[numf])

X_train_sc = scale_numf(X_train, numf, scaler)
X_train_sc.std(axis=0)

In [ ]:
X_test_sc = scale_numf(X_test, numf, scaler)

X_train_sc.shape
X_test_sc.shape
X_test_sc.std(axis=0)

## Defining pytorch datasets and dataloaders

In [ ]:
trainds = RegressionColumnarDataset(X_train_sc, catf, y_train)
valds = RegressionColumnarDataset(X_test_sc, catf, y_test)

In [ ]:
params = {'batch_size': 128,
          'shuffle': True,
          'num_workers': 8}

traindl = data.DataLoader(trainds, **params)
valdl = data.DataLoader(valds, **params)

## Defining model and related variables

In [ ]:
y_range = (0, y_train.max()*1.2)
y_range

cat_sz = [(c, df[c].max()+1) for c in catf]
cat_sz

emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
emb_szs

In [ ]:
m = MixedInputModel(emb_szs=emb_szs, 
                    n_cont=len(df.columns)-len(catf), 
                    emb_drop=0.04, 
                    out_sz=1, 
                    szs=[1000,500,250], 
                    drops=[0.001,0.01,0.01], 
                    y_range=y_range).to(device)

opt = optim.Adam(m.parameters(), 1e-2)
lr_cosine = lr_scheduler.CosineAnnealingLR(opt, 1000)

lr = defaultdict(list)
tloss = defaultdict(list)
vloss = defaultdict(list)

In [ ]:
m

## Training the model

In [ ]:
lr, tloss, vloss = fit(model=m, train_dl=traindl, val_dl=valdl, loss_fn=F.mse_loss, opt=opt, scheduler=lr_cosine, epochs=10)

In [ ]:
t = [np.mean(tloss[el]) for el in tloss]
v = [np.mean(vloss[el]) for el in vloss]
p = pd.DataFrame({'Train Loss': t, 'Validation Loss': v, 'Epochs': range(10)})

_ = p.plot(x='Epochs', y=['Train Loss', 'Validation Loss'], 
           title='Train and Validation Loss over Epochs')

## Write Model to File

In [ ]:
model_path = os.path.join('./modelfiles/', 'model.pkl')
torch.save(m, model_path)

## Register Model with AML

In [ ]:
model = Model.register(model_path = model_path,
                     model_name = "pytorch_regression",
                     tags = {"onnx": "demo"},
                    description = "Taxi PyTorch Regression",
                    workspace = ws)
